In [1]:
import pathlib, os, sys

In [2]:
# Update PythonPath to resolve imports from src/
src_dir = str(pathlib.Path(os.getcwd()).parent.parent.absolute())
sys.path.append(src_dir)
src_dir

'/Users/divi/Desktop/RosettaCV/src'

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
from models.CMC.contrastive_multiview import *
from models.CMC.ResNetEncoder import ResNetEncoder
from data_loader.MultiviewDatasets import *
from utils.util import split_dataset
from losses.cmc_losses import contrastive_loss
from models.CMC.contrastive_multiview import CMCTrainer, View, WrapperModel
from models.CMC.language_models import TextEncoder

In [5]:
path2data = "../../data/train2014"
path2json = "../../data/annotations/captions_train2014.json"
train_loader, val_loader = get_coco_data_loaders(path2data, path2json, 32, 0.7)

loading annotations into memory...
Done (t=0.64s)
creating index...
index created!


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fe1 = ResNetEncoder(device, latent_dim=512)
lm1 = TextEncoder(latent_dim=512)

In [7]:
view1 = View(fe1)
textview = View(lm1)
model = WrapperModel([view1, textview], 512)

In [8]:
sample = next(iter(train_loader))

In [9]:
sample[1]

('Two elephants are sitting on large stools at the Circus.',
 'cows chained to some poles while eating some grass in the barn',
 'A bus parked on a street in front of buildings.',
 'a young boy jumps on his skateboard while his friend watches',
 'An overhead view of a bridge crossing water with numerous ships in it.',
 'A duck is standing outdoors looking over a wall. ',
 'a lady blowing out candles on a cake ',
 'A boy in blue shirt opening a large box of pizza.',
 'Another serious Selfie is taken by this serious looking man.',
 'a man about to swing at a baseball at a game',
 'Two men are holding a trophy and smiling.',
 'a group of photos of people riding motorcycles ',
 'A cake on a plate has just been sliced.',
 'A green plant with leaves and very thick stems.',
 'Clock faces displayed in ornate grey brick work.',
 'a bed room with a large window and a neatly made bed',
 'A living room with lit candles on the table and sill',
 'An old truck parked on the side of a road in a rural 

In [10]:
trainer = CMCTrainer(model, contrastive_loss, train_loader, val_loader, num_decodings_per_step=0)
trainer.train(500, save_to='baseline.p')

wandb: Currently logged in as: cal-launchpad (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.28 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


ValueError: too many values to unpack (expected 2)